In [1]:
#Initial Setup Cell
!pip install git+https://github.com/openai/point-e -q
!pip install pandas -q
!pip install open3d -q
!pip install numpy-stl -q
!pip install trimesh -q
!pip install flask -q
!pip install pyngrok -q

import trimesh
from stl import mesh, stl
import pandas as pd
import open3d as o3d
import pandas as pd
import numpy as np
import os
import threading
import open3d as o3d
import requests
import base64
from flask import Flask, request, redirect
from pyngrok import ngrok
from pyngrok import conf
import torch
from tqdm.auto import tqdm
from point_e.util.pc_to_mesh import marching_cubes_mesh
from point_e.diffusion.configs import DIFFUSION_CONFIGS, diffusion_from_config
from point_e.diffusion.sampler import PointCloudSampler
from point_e.models.download import load_checkpoint
from point_e.models.configs import MODEL_CONFIGS, model_from_config
from point_e.util.plotting import plot_point_cloud

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print('creating base model...')
base_name = 'base40M-textvec'
base_model = model_from_config(MODEL_CONFIGS[base_name], device)
base_model.eval()
base_diffusion = diffusion_from_config(DIFFUSION_CONFIGS[base_name])

print('creating upsample model...')
upsampler_model = model_from_config(MODEL_CONFIGS['upsample'], device)
upsampler_model.eval()
upsampler_diffusion = diffusion_from_config(DIFFUSION_CONFIGS['upsample'])

print('downloading base checkpoint...')
base_model.load_state_dict(load_checkpoint(base_name, device))

print('downloading upsampler checkpoint...')
upsampler_model.load_state_dict(load_checkpoint('upsample', device))

sampler = PointCloudSampler(
      device=device,
      models=[base_model, upsampler_model],
      diffusions=[base_diffusion, upsampler_diffusion],
      num_points=[1024, 4096-1024],
      aux_channels=['R', 'G', 'B'],
      guidance_scale=[3.0, 0.0],
      model_kwargs_key_filter=('texts', ''), # Do not condition the upsampler at all
)

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
creating base model...
creating upsample model...
downloading base checkpoint...
downloading upsampler checkpoint...


In [ ]:
import hmac
import hashlib
from flask import jsonify

GITHUB_WEBHOOK_SECRET = 'yellowclaw'

# Global server flag
server_busy = False

# Constants
TOKEN = 'TOKEN'
OWNER = 'gabrielramp'
REPO = 'hackgtsecret'
BRANCH = 'main'
FILE_PATH = 'README.md'
COMMIT_MESSAGE = 'Update README.md with pizzaz!'
HEADERS = {
    'Authorization': f'token {TOKEN}',
    'Accept': 'application/vnd.github.v3+json'
}

def generateReadmeModel(prompt: str):
    # Produce the model.
    samples = None
    for x in tqdm(sampler.sample_batch_progressive(batch_size=1, model_kwargs=dict(texts=[prompt]))):
        samples = x
    pc = sampler.output_to_point_clouds(samples)[0]
    pc.save('pointcloud.npz')

    # Convert the point cloud into a mesh.
    print('creating SDF model...')
    name = 'sdf'
    model = model_from_config(MODEL_CONFIGS[name], device)
    model.eval()

    print('loading SDF model...')
    model.load_state_dict(load_checkpoint(name, device))

    # Produce a mesh
    mesh = marching_cubes_mesh(
        pc=pc,
        model=model,
        batch_size=4096,
        grid_size=128,
        progress=True,
    )

    with open('mesh.ply', 'wb') as f:
        mesh.write_ply(f)

    # Convert and simplify the mesh
    o3d_mesh = o3d.io.read_triangle_mesh('mesh.ply')
    decimation_target_number_of_triangles = int(0.01 * len(o3d_mesh.triangles))
    simplified_mesh = o3d_mesh.simplify_quadric_decimation(decimation_target_number_of_triangles)
    simplified_mesh.compute_vertex_normals()
    o3d.io.write_triangle_mesh('simplified_output_mesh.stl', simplified_mesh)
    print('done')

    # Convert to ASCII STL
    simplified_mesh_trimesh = trimesh.load('simplified_output_mesh.stl')
    simplified_mesh_trimesh.export(file_obj='ascii_simplified_output_mesh.stl', file_type='stl_ascii')
    print('done')

    # Update the README
    with open('ascii_simplified_output_mesh.stl', 'r') as f:
        stl_content = f.read()

    content = f"Prompt: {prompt} \n```stl\n{stl_content}\n``` \n# 3DREADME \nIs a text-to-model generative AI tool built entirely within GitHub! 3DREADME will take any prompt and create a 3D model for the described object using point-cloud to ASCII STL conversion. \n\n## How do I use it?\nCreate a new issue in this repository through this link:\n\nhttps://github.com/gabrielramp/hackgtsecret/issues/new\n\nTitle the issue the text prompt you wish to be created into a model and displayed on this README.md. No body text is required in the new issue. You will automatically receive a response when the request is received, and the issue will be closed once this README is update with your model!\n\nThe model generator works best with simple objects, i.e 'a motorcycle', 'an airplane', or 'a laptop'.\n\n## How does it work?\n\n3DREADME first leverages GitHub's Webhooks and Issues to receive a text prompt from the user (you!). We then utilize a tweaked version OpenAI's Point-E model to generate a point-cloud field that represents the described object.\n\n![image](https://github.com/gabrielramp/hackgtsecret/assets/86631042/9f50ac08-2ce5-4111-8638-524ad337f219)\n\nThen, we estimate the normals of the verticies in the point-cloud by assessing their proximity to each other, and utilize the Ball Pivoting Algorithm to create faces in the mesh that results in a solid object.\n\n![image](https://github.com/gabrielramp/hackgtsecret/assets/86631042/a9f85174-6703-4c99-8c4c-393c76e815df)\n(Image provided by IEEE.)\n\nAfter utilizing GitHub Webhooks to listen for Issue events, and the GitHub API to reply, update the README, and close your issue, the result is a water-tight representation of your text description displayed right in the README of the repository thanks to GitHub Diagrams!"
    update_readme(content)

def update_readme(content):
    try:
        response = requests.get(f'https://api.github.com/repos/{OWNER}/{REPO}/contents/{FILE_PATH}', headers=HEADERS)
        response.raise_for_status()
        sha = response.json()['sha']
        encoded_content = base64.b64encode(content.encode('utf-8')).decode('utf-8')
        update_data = {
            'message': COMMIT_MESSAGE,
            'content': encoded_content,
            'sha': sha,
            'branch': BRANCH
        }
        response = requests.put(f'https://api.github.com/repos/{OWNER}/{REPO}/contents/{FILE_PATH}', json=update_data, headers=HEADERS)
        response.raise_for_status()
        print('README.md updated successfully!')
    except requests.RequestException as error:
        print(f'Error updating the README.md: {error}')

def is_valid_signature(payload, signature):
    secret_token = GITHUB_WEBHOOK_SECRET.encode()
    computed_hmac = hmac.new(secret_token, payload, hashlib.sha1)
    expected_signature = f"sha1={computed_hmac.hexdigest()}"
    return hmac.compare_digest(expected_signature, signature)

def reply_to_issue(issue_number, message):
    url = f"https://api.github.com/repos/{OWNER}/{REPO}/issues/{issue_number}/comments"
    data = {
        'body': message
    }
    response = requests.post(url, json=data, headers=HEADERS)
    response.raise_for_status()

def close_issue(issue_number):
    url = f"https://api.github.com/repos/{OWNER}/{REPO}/issues/{issue_number}"
    data = {
        'state': 'closed'
    }
    response = requests.patch(url, json=data, headers=HEADERS)
    response.raise_for_status()

# Spin up the Ngrok server
conf.get_default().auth_token = "TOKEN"

app = Flask(__name__)
port = "5000"

# Open a ngrok tunnel to the HTTP server
public_url = ngrok.connect(port, domain="threedeereadme.ngrok.dev").public_url
print(" * ngrok tunnel \"{}\" -> \"http://127.0.0.1:{}\"".format(public_url, port))

# Update any base URLs to use the public ngrok URL
app.config["BASE_URL"] = public_url

# Define Flask routes
@app.route("/", methods=['GET', 'POST'])
def index():
    global server_busy

    if request.method == 'POST':
        user_input = request.form.get('text_input')
        # Process the input as needed
        print('received request for', user_input)

        # Check if the server is already busy
        if server_busy:
            return "The server is busy! Please try again soon."

        # Set the server state as busy
        server_busy = True

        try:
            generateReadmeModel(user_input)
            return redirect("https://github.com/gabrielramp/hackgtsecret")
            server_busy = False
        finally:
            # Reset the server state back to not busy
            server_busy = False

    return '''
        <form method="post" action="/">
            <input type="text" name="text_input">
            <input type="submit" value="Enter">
        </form>
    '''
@app.route("/webhook", methods=["POST"])
def handle_github_webhook():
    global server_busy

    # Verify if the incoming webhook is from GitHub
    signature = request.headers.get("X-Hub-Signature")
    if not signature or not is_valid_signature(request.data, signature):
        return jsonify({"message": "Invalid request"}), 400

    # Get the payload data
    data = request.json

    # Check if the event is an issue and it's opened
    if request.headers.get("X-GitHub-Event") == "issues" and data["action"] == "opened":
        issue_title = data["issue"]["title"]
        issue_number = data["issue"]["number"]
        reply_to_issue(issue_number, "Request received -- Please wait!")

        # Check if the server is already busy
        if server_busy:
            reply_to_issue(issue_number, "The server is currently busy. Please try again soon!")
            close_issue(issue_number)
            return jsonify({"message": "The server is busy! Please try again soon."}), 503

        # Set the server state as busy
        server_busy = True

        try:
            generateReadmeModel(issue_title)
            reply_to_issue(issue_number, "README.md updated, closing ticket!")
            close_issue(issue_number)
            return jsonify({"message": "Processed successfully!"}), 200
        finally:
            # Reset the server state back to not busy
            server_busy = False


# Start the Flask server in a new thread
threading.Thread(target=app.run, kwargs={"use_reloader": False}).start()

#DEBUG force generation on cell start
#generateReadmeModel("a spaceship")